# Definition and training of the ConvNet computational graph

## Matching the models' "expressivities"

The idea of ConvNet is to use a more common structure while keeping approximately the same expressivity as SpotNet. In this way, the improvement in performance of SpotNet over ConvNet can really be attributed to its beneficial structure. To achieve this goal, the total number of learnable parameters should be approximately the same in both models. The following

| Model | # learnable $5\times 5$ conv. kernels | # scalar parameters |
| --    | --                                      | --                  |
| SpotNet | 210 | 8 |
| ConvNet | 216 | 6 |
|<td colspan=3> <center> Tab. 1: Comparison of the number of learnable parameters in SpotNet and ConvNet. Separated into the number of learnable $5\times5$ convolutional kernels and the number of learnable scalar parameters. For details on how these numbers were obtained, see the following sections. </center> |

### Analysis of SpotNet in terms of number of parameters

SpotNet has $30$ convolutions in its input half-layer, and $60$ convolutions in each hidden layer (of which there are $3$). This amounts to a total of $30+60\times 3=\mathbf{210}$ two-dimensional convolutions with $5\times 5$ learnable kernels. Besides the learnable kernels, SpotNet has $1$ scalar parameter and $1$ one-dimensional bias parameter per each of its $3$ layers, and $1$ more of each for its input half-layer, amounting to a total of $4\times 2 = \mathbf{8}$ scalar paremeters.

<a id="design"></a>
### Design of ConvNet

A full convolutional layer with $N$ input channels, $M$ output channels, and $5 \times 5$ kernels implies $N \times M$ two-dimensional convolutions with such kernels. Similarly, a convolutional layer that turns each of the $N$ input channels into $K$ contains $N\times K$ two-dimensional convolutions. Furthermore, a feature-wise convolutional layer involves only $1$ two-dimensional convolution per feature. Thereby, the structure we propose for ConvNet, i.e., 

1. $2$ full convolutional layers to turn the single input into $6$ features and, succesively, the $6$ features into $15$, 
2. $1$ layer that turns each feature into two, and 
3. $3$ layers with feature-wise convolutions,   

results in a grand total of $1\times  6 + 6 \times 15 + 15 \times 2 + 30 + 30 +30 = \mathbf{216}$ two-dimensional convolutions with $5\times 5$ learnable kernels. Furthermore, this structure we propose for ConvNet has $1$ one-dimensional bias term for each layer, and a total of $6$ layers, amounting ot $1\times 6 = \mathbf{6}$ scalar parameters. 


# Index
1. Definition of computational graph of ConvNet
    1. [Importing libraries](#libs) to build and initialize the network
    2. Definition of the [full ConvNet computational graph](#convnet)
    6. Definition of the [training strategy](#training_strategy) for ConvNet
2. Training of ConvNet (not recommended for non-GPU systems)
    1. [Loading of the training data-base](#data) and setting of parameters 
    2. Construction of the [computational graph](#session)
    3. Running the [training](#training) and storing its results

# Definition of computational graph of ConvNet

<a id="libs"></a>
## Importing libraries to build and initialize the network

In [1]:
import tensorflow as tf
import numpy as np

<a id="convnet"></a>
## Definition of the full ConvNet computational graph

To compare with the [Design of ConvNet](#design) section above.

In [2]:
def conv_net( image_height = 512, image_width = 512 ):
    
    # Seed for reproducibility
    tf.set_random_seed( 8888 )
    
    # Input placeholder
    source = tf.placeholder( tf.float32, [ 1, image_height, image_width ] )
    
    # First convolutional layer, from source to 6 channels
    h1 = tf.Variable( tf.truncated_normal( [5, 5, 1, 6], stddev = 0.01 ) )
    x1 = tf.nn.conv2d( tf.expand_dims( source, axis = 3 ), h1, strides = [1, 1, 1, 1], padding = 'SAME' )
    b1 = tf.Variable( 1.0 )
    x1 = tf.nn.relu(  x1 + b1 )
    
    # Second convolutional layer, from 6 to 15 channels
    h2 = tf.Variable( tf.truncated_normal( [5, 5, 6, 15], stddev = 0.01 ) )
    x2 = tf.nn.conv2d( x1, h2, strides = [1, 1, 1, 1], padding = 'SAME' )
    b2 = tf.Variable( 1.0 )
    x2 = tf.nn.relu( x2 + b2 )
    
    # Third convolutional layer, each of the 15 channels splits in 2
    h3 = tf.Variable( tf.truncated_normal( [5, 5, 15, 2], stddev = 0.01 ) )
    x3 = tf.nn.depthwise_conv2d( x2, h3, strides = [1, 1, 1, 1], padding = 'SAME' )
    b3 = tf.Variable( 1.0 )
    x3 = tf.nn.relu( x3 + b3  )
    
    x = [x3]; b = []; h = [];
    # Three layers in which each channel is treated independently, i.e. from 30 channels to 30 channels
    for layer_number in range(3):
        h_next = tf.Variable( tf.truncated_normal( [5, 5, 30, 1], stddev = 0.01 ) )
        x_next = tf.nn.depthwise_conv2d( x[-1], h_next, strides = [1, 1, 1, 1], padding = 'SAME' )
        b_next = tf.Variable( 1.0 )
        x_next = tf.nn.relu( x_next + b_next )
        x.append( x_next )
        b.append( b_next )
        h.append( h_next )    
    
    # Output is the result at the end of the last layer
    output = x[-1]
    
    # Placeholder for the target
    target = tf.placeholder( tf.float32, [ 1, image_height, image_width, 30 ] )
    
    # MSE loss function
    loss = tf.reduce_mean( ( output - target ) ** 2 )
    
    return (source, loss, target, output)


<a id="training_strategy"></a>
## Definition of the training strategy for SpotNet

In [ ]:
# Create and return the optimizer
def network_training( loss, learning_rate ):
    return tf.train.AdamOptimizer( learning_rate ).minimize( loss )

<a id="training_sec"></a>
# Training of SpotNet

<a id="data"></a>
## Loading of the training data-base and setting of parameters

In [3]:
# Learning process parameters
batch_size = 1
learning_rate = 1e-3
nrof_train_steps = 400000
nrof_cells = 1250

# Relevant directories
results_dir = 'results/'
data_dir = 'sim_data/'

# Load dataset
data = np.load( data_dir + 'result_' + str( nrof_cells ) + '_cells_10_images.npy' )[()]

# Extract images and xs
images = data['fluorospot']
psdrs = data['psdrs']

# Extract shape parameters
nrof_images, image_height, image_width = images.shape
_, _, _, nrof_kernels = psdrs.shape

# Split dataset for training and validation
nrof_training_samples = int( 0.7 * nrof_images )
train_images, train_psdrs = (images[ : nrof_training_samples, ... ], psdrs[ : nrof_training_samples, ... ])
val_images, val_psdrs = (images[nrof_training_samples : , ... ], psdrs[nrof_training_samples : , ... ])

# Inform user
print( 'Dataset: Num images: %d, Image height: %d, Image width: %d, Num cells: %d, Num kernels: %d'%( nrof_images, 
                                                                                                      image_height, 
                                                                                                      image_width,
                                                                                                      nrof_cells,
                                                                                                      nrof_kernels ) )

Dataset: Num images: 10, Image height: 512, Image width: 512, Num cells: 1250, Num kernels: 30


<a id="session"></a>
## Construction of the computational graph

In [4]:
# Build computational graph
with tf.name_scope( 'ConvNet' ) as scope:
    source, loss, target, output = conv_net( image_height, image_width )

# Build training computational graph
with tf.name_scope( 'Training' ) as scope:
    train_step = network_training( loss, learning_rate )
    
# Create TensorFlow session to run the computational graph
nrof_gpu = 1
config = tf.ConfigProto( device_count = {'GPU': nrof_gpu} )
sess = tf.Session( config = config )
sess.run( tf.global_variables_initializer( ) )
print( "An error will be issued here if no GPU is present in the system. This is intentional.\n" + 
       "Training is not recommended for non-GPU systems. Set 'nrof_gpus = 0' above if you still want to proceed." )

<a id="training"></a>
## Running training and storing its results

In [6]:
# Set TensorFlow logging level to non-debug (only WARN and ERROR log messages)
tf.logging.set_verbosity( tf.logging.WARN )

# Time library to time training
import time

# Inform user
print( 'Training the network' )

# Space to store loss values
train_loss_record = np.empty( (0, ) )
val_loss_record  = np.empty( (0, ) )
iterations_record = np.empty( (0, ) )

# Indices that will be picked at random at each iteration
indices = np.arange( train_images.shape[0] )

# Seed for reproducibility
np.random.seed( 8888 )

# Train the network
start = time.time( )
for training_iteration in range( nrof_train_steps + 1 ):
    # Shuffle indices
    np.random.shuffle( indices )
    # Extract current batch
    batch_indices = np.take( indices, np.arange( batch_size ), mode = 'wrap' )
    batch_input = train_images[ batch_indices, ... ]
    batch_target = train_psdrs[ batch_indices, ... ]
    # Run a training step
    _, train_loss = sess.run( [ train_step, loss ], feed_dict = { source: batch_input, 
                                                                  target: batch_target } )
    
    # Every 100th of total iterations, record progress in terms of validation loss
    if training_iteration % (nrof_train_steps / 100) == 0:
        # Sum validation losses
        val_loss = 0
        for val_image_index in range( val_images.shape[0] ):
            val_loss += sess.run( loss, feed_dict = { source: np.expand_dims( val_images[ val_image_index, ... ], axis = 0 ), 
                                                       target: np.expand_dims( val_psdrs[  val_image_index, ... ], axis = 0 ) } )
        # Divide by number of validation images
        val_loss = val_loss / val_images.shape[0]
        # Store losses
        val_loss_record   = np.append( val_loss_record, val_loss )
        train_loss_record = np.append( train_loss_record, train_loss )
        iterations_record = np.append( iterations_record, training_iteration )
        # Inform the user and store the model if it is the best one yet
        if (val_loss == val_loss_record.min() and training_iteration > 10):
            print( 'Train step %d, Batch loss: %0.4f, Test loss: %0.4f, Elapsed: %ds. Best in test yet! Storing.'%( 
                                                                                    training_iteration, 
                                                                                    train_loss, 
                                                                                    val_loss, 
                                                                                    time.time() - start ) ) 
            tf.saved_model.simple_save( sess,
                results_dir + 'trained_convnet_' + str( training_iteration ) + '_train-steps_5_kersize/',
                inputs = {'image': source},
                outputs = {'psdr': output} )
        else:
            print( 'Train step %d, Batch loss: %0.4f, Test loss: %0.4f, Elapsed: %ds.'%( 
                                                                                    training_iteration, 
                                                                                    train_loss, 
                                                                                    val_loss, 
                                                                                    time.time() - start ) )
# Store loss values
# Store loss values
np.savez( results_dir + 'trained_convnet_history_train-steps_5_kersize', train_loss_record = train_loss_record, 
                                                                         val_loss_record   = val_loss_record, 
                                                                         iterations_record = iterations_record )



Training the network
Train step 0, Batch loss: 2321.3057, Test loss: 1590.5524, Elapsed: 3s.
Train step 4000, Batch loss: 1452.4852, Test loss: 1426.8415, Elapsed: 1964s. Best in test yet! Storing.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_4000_train-steps_5_kersize/saved_model.pb
Train step 8000, Batch loss: 1881.9786, Test loss: 1394.3916, Elapsed: 3909s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_8000_train-steps_5_kersize/saved_model.pb
Train step 12000, Batch loss: 1413.0874, Test loss: 1388.3988, Elapsed: 5853s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/

INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_140000_train-steps_5_kersize/saved_model.pb
Train step 144000, Batch loss: 1390.3242, Test loss: 1002.9410, Elapsed: 69421s.
Train step 148000, Batch loss: 1041.9705, Test loss: 992.8294, Elapsed: 71390s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_148000_train-steps_5_kersize/saved_model.pb
Train step 152000, Batch loss: 1095.2675, Test loss: 985.7089, Elapsed: 73346s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_152000_train-steps_5_kersize/saved_model.pb
Train step 156000, Batch loss: 1004.6012, Test loss: 994.3376, Elapsed: 75319s.
Train step 160000, Batch loss: 1284.1036, Test loss: 990.7282, Elapsed: 77294s.
Train step 164